In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### **Markets Dataset**

In [2]:
df_markets = pd.read_csv("sales_markets.csv")
df_markets.tail()

,markets_code,markets_name,zone
12,Mark013,Bhopal,Central
13,Mark014,Hyderabad,South
14,Mark015,Bhubaneshwar,South
15,Mark097,New York,NaN
16,Mark999,Paris,NaN


In [3]:
df_markets.shape

(17, 3)

#### **If data is null in zone column droping that row completly**

In [4]:
df_markets.dropna(subset=["zone"], axis = 0, inplace=True)

In [5]:
df_markets.tail()

,markets_code,markets_name,zone
10,Mark011,Nagpur,Central
11,Mark012,Surat,North
12,Mark013,Bhopal,Central
13,Mark014,Hyderabad,South
14,Mark015,Bhubaneshwar,South


### **Transactions Dataset**

In [6]:
df_transactions = pd.read_csv("sales_transactions.csv")
df_transactions.head()

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency
0,Prod001,Cus001,Mark001,2017-10-10,100,41241.0,INR
1,Prod001,Cus002,Mark002,2018-05-08,3,-1.0,INR
2,Prod002,Cus003,Mark003,2018-04-06,1,875.0,INR
3,Prod002,Cus003,Mark003,2018-04-11,1,583.0,INR
4,Prod002,Cus004,Mark003,2018-06-18,6,7176.0,INR


#### **Drop rows where sales_amount is -1 or 0**

In [7]:
print("Before tansformation;\n")
print("Count of -1 datas in sales_amount column : {}".format(len(df_transactions[df_transactions["sales_amount"] == -1])))
print("Count of 0 datas in sales_amount column : {}".format(len(df_transactions[df_transactions["sales_amount"] == 0])))

df_transactions.drop(df_transactions[df_transactions["sales_amount"] == -1].index, inplace=True)
df_transactions.drop(df_transactions[df_transactions["sales_amount"] == 0].index, inplace=True)
print("\nAfter tansformation;\n")

print("Count of -1 datas in sales_amount column : {}".format(len(df_transactions[df_transactions["sales_amount"] == -1])))
print("Count of 0 datas in sales_amount column : {}".format(len(df_transactions[df_transactions["sales_amount"] == 0])))


Before tansformation;

Count of -1 datas in sales_amount column : 2
Count of 0 datas in sales_amount column : 1609

After tansformation;

Count of -1 datas in sales_amount column : 0
Count of 0 datas in sales_amount column : 0


In [8]:
df_transactions.head()

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency
0,Prod001,Cus001,Mark001,2017-10-10,100,41241.0,INR
2,Prod002,Cus003,Mark003,2018-04-06,1,875.0,INR
3,Prod002,Cus003,Mark003,2018-04-11,1,583.0,INR
4,Prod002,Cus004,Mark003,2018-06-18,6,7176.0,INR
5,Prod003,Cus005,Mark004,2017-11-20,59,500.0,USD


#### **Drop rows where currency is USD or INR because those datas are replicated. And rename USD\r and INR\r with just USD and INR**

In [9]:
print("Before tansformation:")
df_transactions["currency"].value_counts()

Before tansformation:


INR\r    148393
INR         275
USD           2
USD\r         2
Name: currency, dtype: int64

In [10]:
df_transactions.drop(df_transactions[df_transactions["currency"] == "INR"].index, inplace=True)
df_transactions.drop(df_transactions[df_transactions["currency"] == "USD"].index, inplace=True)
df_transactions.reset_index(drop=True,inplace=True)
df_transactions["currency"] = df_transactions["currency"].apply(lambda x: "USD" if x == "USD\r" else "INR")

print("After tansformation:")
df_transactions["currency"].value_counts()

After tansformation:


INR    148393
USD         2
Name: currency, dtype: int64

#### **And also apply rates as given on sales_amount where currency is USD\r**

In [11]:
df_dict = {"Year" : [2017,2018,2019,2020],
           "USD/INR" : [65.0966,68.4113,70.4059,74.1322]}
df = pd.DataFrame(data = df_dict)
df.head()

,Year,USD/INR
0,2017,65.0966
1,2018,68.4113
2,2019,70.4059
3,2020,74.1322


In [12]:
df_transactions["sales_amount"] = df_transactions.apply(lambda x: x["sales_amount"]*df[df["Year"] == int(df_transactions["order_date"][5][:4])].values[0][1] if x["currency"] == 'USD' else x["sales_amount"], axis=1)

In [13]:
# df_transactions["sales_amount"] = df_transactions.apply(lambda x: x["sales_amount"]*75 if x["currency"] == 'USD' else x["sales_amount"], axis=1)
# başarılı fakat daha iyisini yaptık yukarıda!!

In [14]:
df_transactions[df_transactions["currency"] == "USD"]

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency
4,Prod003,Cus005,Mark004,2017-11-20,59,32548.30,USD
5,Prod003,Cus005,Mark004,2017-11-22,36,16274.15,USD


### **Products Dataset**

In [7]:
df_products = pd.read_csv("sales_products.csv")
df_products.head()

,product_code,product_type
0,Prod001,Own Brand\r
1,Prod002,Own Brand\r
2,Prod003,Own Brand\r
3,Prod004,Own Brand\r
4,Prod005,Own Brand\r


In [8]:
df_products["product_type"].value_counts()

Own Brand\r       191
Distribution\r     88
Name: product_type, dtype: int64

In [9]:
df_products["product_type"] = df_products["product_type"].apply(lambda x: "Own Brand" if x == "Own Brand\r" else "Distribution")

In [10]:
df_products["product_type"].value_counts()

Own Brand       191
Distribution     88
Name: product_type, dtype: int64

In [15]:
df_transactions.to_csv("deneme/sales_transactions.csv", index=False)

In [16]:
df_markets.to_csv("deneme/sales_markets.csv", index=False)

In [11]:
df_products.to_csv("deneme/sales_products.csv", index=False)